In [29]:
# Step 1: Import Libraries and Load the Model
import numpy as np
import pandas as pd
import pickle
import os
import nselib
from nselib import capital_market

import tensorflow as tf

from keras.preprocessing import sequence
from keras.models import load_model

In [ ]:


# def fetch_and_save_nse_data(symbol, start_date='01-01-2021', end_date='30-09-2025', save_path=None):
#     # Fetch data
#     data = capital_market.price_volume_data(symbol=symbol, from_date=start_date, to_date=end_date)
    
#     # Filter EQUITY SERIES only
#     data_eq = data[data['Series'] == 'EQ']
    
#     # Select relevant columns
#     cols = ['Symbol', 'Series', 'Date', 'PrevClose', 'OpenPrice', 'HighPrice',
#             'LowPrice', 'ClosePrice', 'TotalTradedQuantity']
#     data_eq = data_eq[cols]
    
#     # Convert columns to float
#     n_cols = ['PrevClose', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice']
#     for col in n_cols:
#         data_eq[col] = data_eq[col].astype(float)
    
#     # Convert volume (remove commas and convert to numeric)
#     data_eq['Volume'] = pd.to_numeric(data_eq['TotalTradedQuantity'].str.replace(',', ''), errors='coerce').astype('Int64')
    
#     # Convert Date column to datetime
#     data_eq['Date'] = pd.to_datetime(data_eq['Date'], format='%d-%b-%Y', errors='coerce')
    
#     # Drop the original TotalTradedQuantity column
#     data_eq = data_eq.drop(columns='TotalTradedQuantity', axis=1)
    
#     print(f'Data preprocessing (`Data Types`) conversion is done...')

#     # Select only Date, ClosePrice, Volume
#     df = data_eq[['Date', 'ClosePrice']]
    
#     # Set Date as index
#     df.set_index('Date', inplace=True)
    
#     # Set default save path if not provided
#     if save_path is None:
#         # Create folder if not exists
#         os.makedirs('Data', exist_ok=True)
#         save_path = f'Data/{symbol}_raw.csv'
    
#     data_eq.to_csv(save_path, index=False, header=True)
#     print(f'Data for {symbol} saved to {save_path}!')

# Example usage:
# fetch_and_save_nse_data('INFY', '02-09-2025', '06-09-2025')


In [ ]:
# fetch_and_save_nse_data('INFY', '01-09-2025', '03-10-2025')


Data preprocessing (`Data Types`) conversion is done...
Data for SBIN saved to Data/SBIN_raw.csv!


In [ ]:
# Load the pre-trained model with ReLU activation
model = load_model('simple_rnn_base.keras')

In [33]:

def predict_next_day_price(latest_data_df, lookback, model_path='simple_rnn_base.keras', scaler_path='scaler.pkl'):
    """
    Predict the next day close price using saved model and scaler.
    
    Parameters:
    - latest_data_df: pandas DataFrame with at least `lookback` rows of latest data,
                      columns must match the features used during training (e.g. ['ClosePrice', 'Volume', ...])
    - lookback: int, how many past timesteps model uses (must be same as training)
    - model_path: str, path to saved model (.h5)
    - scaler_path: str, path to saved scaler (.pkl)
    
    Returns:
    - next_day_pred: float, predicted next day ClosePrice (unscaled)
    """
    
    # 1. Load scaler and model
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)
    model = load_model(model_path)
    
    # 2. Extract last `lookback` rows as numpy array (shape: (lookback, n_features))
    data_array = latest_data_df.values[-lookback:]
    
    # 3. Scale input data using the saved scaler
    data_scaled = scaler.transform(data_array)
    
    # 4. Reshape to (1, lookback, n_features) as model input
    input_seq = data_scaled.reshape(1, lookback, data_scaled.shape[1])
    
    # 5. Predict scaled output (shape: (1, horizon))
    pred_scaled = model.predict(input_seq)
    
    # 6. Inverse scale the prediction
    # Since scaler expects full feature vector, we create a dummy array filled with last known features
    n_features = data_scaled.shape[1]
    dummy = np.tile(data_scaled[-1, :], (pred_scaled.shape[1], 1))  # shape: (horizon, n_features)
    
    # Replace the target feature (assumed index 0 = ClosePrice) with predictions
    dummy[:, 0] = pred_scaled.flatten()
    
    # Inverse transform back to original scale
    pred_inverse = scaler.inverse_transform(dummy)
    
    # Extract just the ClosePrice column for the first horizon step (next day)
    next_day_pred = pred_inverse[0, 0]
    
    return next_day_pred




In [34]:
# Example usage:
# Assume `df` is your full dataframe with features ['ClosePrice', 'Volume', ...] indexed by date
df = pd.read_csv(r'Data\SBIN_raw.csv')
lookback = 20
latest_data = df[['ClosePrice']].iloc[-lookback:]  # match training features

predicted_close = predict_next_day_price(latest_data, lookback)
print(f"Predicted next day ClosePrice: {predicted_close:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 630ms/step
Predicted next day ClosePrice: 868.44
